In [1]:
import tensorflow as tf
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from os import listdir
import glob
from sklearn.utils import shuffle


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
nrClasses = 10
imgWidth = 28
imgHeight = 28
nrChannels = 1
imgPath = '/notebooks/mnist_1000/'
nrImgs = len(glob.glob(imgPath + '/**/*.png', recursive=True))
images = np.empty(shape=[nrImgs, imgWidth, imgHeight, nrChannels]) 
labels = np.zeros(shape=[nrImgs, nrClasses])

instanceCount = 0
for i in range(0, nrClasses):
    path = imgPath + str(i) + '/'
    nrImgCurrentClass = len(listdir(imgPath + str(i) + '/'))
    imgCurrentClass = np.array([mpimg.imread(path+f) for f in listdir(imgPath + str(i) + '/')]) 
    imgCurrentClass = np.array([imgCurrentClass])
    imgCurrentClass = imgCurrentClass.reshape((nrImgCurrentClass, imgWidth, imgHeight, nrChannels)) 
    firstPos = instanceCount
    lastPos = instanceCount + imgCurrentClass.shape[0]
images[firstPos : lastPos] = imgCurrentClass
labels[firstPos : lastPos, i] = 1
instanceCount = lastPos
learnRate = 0.01
nrEpochs = 100
ins = tf.placeholder(shape=[None, imgWidth, imgHeight, nrChannels], dtype=tf.float32, name="in") 
desOuts = tf.placeholder(shape=[None, nrClasses], dtype=tf.float32, name="desouts")
conv1 = tf.layers.conv2d(ins, 3, 5, activation=tf.nn.relu, name="conv1")
pool1 = tf.layers.max_pooling2d(conv1, 2, 2, name="pool1")
fc1 = tf.contrib.layers.flatten(pool1)
fc1 = tf.layers.dense(fc1, 100, activation=tf.nn.sigmoid, name="fc1")
predOuts_logits = tf.layers.dense(fc1, nrClasses, activation=tf.nn.sigmoid)
predOuts = tf.nn.softmax(predOuts_logits)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = desOuts, logits = predOuts_logits)
loss = tf.reduce_mean(cross_entropy)
ms = tf.losses.mean_squared_error(desOuts, predOuts)
tf.summary.scalar('loss', loss)
tf.summary.scalar('square_error', ms)

optimizer = tf.train.AdamOptimizer(learnRate).minimize(loss)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        prediction = tf.argmax(predOuts, 1)
        correct_prediction = tf.equal(prediction, desOuts)
    with tf.name_scope('accuracy'):
        evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', evaluation_step)
merged = tf.summary.merge_all()


TypeError: Input 'y' of 'Equal' Op has type float32 that does not match type int64 of argument 'x'.

In [ ]:
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter('/tmp/tensorflow/mnist/train',
                                      sess.graph)
    test_writer = tf.summary.FileWriter('/tmp/tensorflow/mnist/test', sess.graph)
    sess.run(tf.global_variables_initializer())
    images, labels = shuffle(images, labels, random_state=0)
    trainData = {ins:images[:5000,:,:,:], desOuts:labels[:5000,:]}
    testData = {ins:images[5000:,:,:,:], desOuts:labels[5000:,:]}
    for i in range(0, nrEpochs):
        if i % 5 == 0:  # Record summaries and test-set accuracy
            lossVal, summary = sess.run([ms, merged], testData)
            test_writer.add_summary(summary, i)
            print(lossVal)
        else:  # Record train set summaries, and train
            lossVal, summary = sess.run([optimizer, merged], trainData)
            train_writer.add_summary(summary, i)
